In [1]:
import pandas as pd
import numpy as np

In [39]:
data = pd.read_csv("learn.csv")

In [40]:
del data["Unnamed: 0"]

In [41]:
df = data.copy()

In [6]:
data = data[["application_id", "loan_limit", "loan_rate", "is_applied", "user_id"]]

In [7]:
data

,application_id,loan_limit,loan_rate,is_applied,user_id
0,566528,20000000.0,19.1,0.0,681184
1,180433,16000000.0,15.0,0.0,623737
2,180433,22000000.0,16.6,0.0,623737
3,1657888,3000000.0,13.5,0.0,752985
4,180433,3000000.0,14.9,0.0,623737
...,...,...,...,...,...
10264381,455157,3000000.0,18.0,0.0,717233
10264382,455157,3000000.0,17.5,0.0,717233
10264383,455157,5000000.0,18.4,0.0,717233
10264384,455157,3000000.0,14.5,0.0,717233


In [8]:
g = data.groupby("application_id").max()

In [11]:
g = g.loc[g.is_applied == 1]

In [14]:
apps = g.index

In [15]:
apps

Int64Index([     14,      17,      28,      34,      36,      37,      38,
                 48,      50,      58,
            ...
            2167776, 2167784, 2167787, 2167798, 2167803, 2167807, 2167811,
            2167817, 2167820, 2167826],
           dtype='int64', name='application_id', length=377506)

In [16]:
data = data.loc[data.application_id.isin(apps)]

In [18]:
istrue = data.loc[data.is_applied == 1]

In [20]:
y = istrue.groupby("application_id").min()

In [25]:
y = y.reset_index("inplace" == True, "drop" == True)

/var/folders/mj/jcmvk9nd451b_mnylnj74qrm0000gn/T/ipykernel_3907/3518936550.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.reset_index except for the argument 'level' will be keyword-only.
  y = y.reset_index("inplace" == True, "drop" == True)


In [26]:
y

,application_id,loan_limit,loan_rate,is_applied,user_id
0,14,10000000.0,6.6,1.0,623310
1,17,9000000.0,7.1,1.0,822111
2,28,5000000.0,6.1,1.0,495556
3,34,16000000.0,17.4,1.0,766892
4,36,10000000.0,10.7,1.0,746778
...,...,...,...,...,...
377501,2167807,16000000.0,9.4,1.0,636572
377502,2167811,12000000.0,8.0,1.0,661020
377503,2167817,1000000.0,19.9,1.0,314169
377504,2167820,4000000.0,14.9,1.0,781452


In [28]:
d = data.groupby("application_id").min()

In [29]:
d = d.reset_index("drop" == True, "inplace" == True)

/var/folders/mj/jcmvk9nd451b_mnylnj74qrm0000gn/T/ipykernel_3907/281886159.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.reset_index except for the argument 'level' will be keyword-only.
  d = d.reset_index("drop" == True, "inplace" == True)


In [30]:
d

,application_id,loan_limit,loan_rate,is_applied,user_id
0,14,9000000.0,6.6,0.0,623310
1,17,9000000.0,7.1,1.0,822111
2,28,3000000.0,6.1,0.0,495556
3,34,16000000.0,17.4,1.0,766892
4,36,1000000.0,10.2,0.0,746778
...,...,...,...,...,...
377501,2167807,3000000.0,9.4,0.0,636572
377502,2167811,11000000.0,6.5,0.0,661020
377503,2167817,1000000.0,18.4,0.0,314169
377504,2167820,4000000.0,13.9,0.0,781452


In [31]:
lr = y.loan_rate

In [33]:
lr2 = d.loan_rate

In [35]:
s = lr == lr2

In [38]:
(len(s) - s.sum())/len(s)

0.3271338733689001

In [43]:
df = df[["application_id", "loan_limit", "loan_rate", "product_id", "is_applied"]]

In [44]:
from sklearn.linear_model import LogisticRegression

In [83]:
model = LogisticRegression(class_weight = "balanced")

In [47]:
df = df[["loan_limit", "loan_rate", "is_applied"]]
df.loan_limit = np.log10(df.loan_limit + 1)

In [50]:
X = df[["loan_limit", "loan_rate"]]
y = df["is_applied"]

In [60]:
from sklearn.model_selection import train_test_split

In [61]:
x1, x2, y1, y2 = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [84]:
model.fit(x1, y1)

LogisticRegression(class_weight='balanced')

In [85]:
model.coef_

array([[-0.26919268, -0.05350142]])

In [86]:
model.get_params()

{'C': 1.0,
 'class_weight': 'balanced',
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 100,
 'multi_class': 'auto',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': None,
 'solver': 'lbfgs',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

In [87]:
p = model.predict(x2)

In [91]:
model.score(x2, y2)

0.6094551161832316

In [78]:
p2 = model.predict_proba(x2)

In [66]:
from sklearn.metrics import f1_score

In [89]:
f1_score(y2, p)

0.12251156590348668

In [92]:
from sklearn.neural_network import MLPClassifier

In [93]:
neural = MLPClassifier(solver='lbfgs', alpha=1e-5,
                    hidden_layer_sizes=(5, 2), random_state=1)

In [94]:
neural.fit(x1, y1)

MLPClassifier(alpha=1e-05, hidden_layer_sizes=(5, 2), random_state=1,
              solver='lbfgs')

In [95]:
neural.score(x2, y2)

0.9457629727631159

In [97]:
p = neural.predict(x2)

In [99]:
p2 = neural.predict_proba(x2)

In [106]:
p2 > 0.054

array([False, False, False, ..., False, False, False])

In [107]:
f1_score(y2, p2 > 0.054)

0.14028022234817975

In [108]:
np.dot(p2 > 0.054, )

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()